# Crisis Support Funnel

This notebook measures how many sessions move through a crisis-support path:

1. Session has at least one page view (landing)
2. Session reaches a crisis-related page
3. Session clicks a call link (`tel:131114`)


In [ ]:
from google.cloud import bigquery
import pandas as pd
import plotly.graph_objects as go

PROJECT_ID = "lifeline-website-480522"
DATASET = "analytics_315584957"
DAYS_BACK = 35

client = bigquery.Client(project=PROJECT_ID)

In [ ]:
import sys
sys.path.insert(0, "..")
import lifeline_theme

lifeline_theme.inject_fonts()

In [ ]:
query = f"""
WITH event_base AS (
  SELECT
    PARSE_DATE('%Y%m%d', event_date) AS event_day,
    event_name,
    user_pseudo_id,
    CONCAT(
      user_pseudo_id,
      '.',
      COALESCE(CAST((
        SELECT ep.value.int_value
        FROM UNNEST(event_params) ep
        WHERE ep.key = 'ga_session_id'
      ) AS STRING), '0')
    ) AS session_key,
    COALESCE((
      SELECT ep.value.string_value
      FROM UNNEST(event_params) ep
      WHERE ep.key = 'source'
    ), '(direct)') AS source,
    COALESCE((
      SELECT ep.value.string_value
      FROM UNNEST(event_params) ep
      WHERE ep.key = 'medium'
    ), '(none)') AS medium,
    COALESCE((
      SELECT ep.value.string_value
      FROM UNNEST(event_params) ep
      WHERE ep.key = 'page_location'
    ), '(unknown)') AS page_location,
    COALESCE((
      SELECT ep.value.string_value
      FROM UNNEST(event_params) ep
      WHERE ep.key = 'link_url'
    ), '') AS link_url
  FROM `{PROJECT_ID}.{DATASET}.events_*`
  WHERE _TABLE_SUFFIX BETWEEN FORMAT_DATE('%Y%m%d', DATE_SUB(CURRENT_DATE(), INTERVAL {DAYS_BACK} DAY))
    AND FORMAT_DATE('%Y%m%d', CURRENT_DATE())
), cleaned AS (
  SELECT
    event_day,
    event_name,
    user_pseudo_id,
    session_key,
    source,
    medium,
    CASE
      WHEN page_location = '(unknown)' THEN '(unknown)'
      ELSE REGEXP_REPLACE(REGEXP_REPLACE(page_location, r'#.*$', ''), r'\?.*$', '')
    END AS page_location_clean,
    link_url
  FROM event_base
), page_paths AS (
  SELECT
    event_day,
    event_name,
    user_pseudo_id,
    session_key,
    source,
    medium,
    CASE
      WHEN page_location_clean = '(unknown)' THEN '(unknown)'
      WHEN REGEXP_CONTAINS(page_location_clean, r'^https?://') THEN COALESCE(NULLIF(REGEXP_EXTRACT(page_location_clean, r'^https?://[^/]+(/.*)$'), ''), '/')
      WHEN STARTS_WITH(page_location_clean, '/') THEN page_location_clean
      ELSE CONCAT('/', page_location_clean)
    END AS page_path,
    link_url
  FROM cleaned
), session_flags AS (
  SELECT
    source,
    medium,
    session_key,
    MAX(CASE WHEN event_name = 'page_view' THEN 1 ELSE 0 END) AS has_landing,
    MAX(CASE
      WHEN event_name = 'page_view'
        AND REGEXP_CONTAINS(page_path, r'^/(get-help|crisis-support|suicide|131114|chat|text)') THEN 1
      ELSE 0
    END) AS has_crisis_page,
    MAX(CASE
      WHEN event_name = 'click'
        AND REGEXP_CONTAINS(LOWER(link_url), r'tel:(\+?61)?131114|13\s*11\s*14') THEN 1
      ELSE 0
    END) AS has_call_click
  FROM page_paths
  GROUP BY source, medium, session_key
)
SELECT
  source,
  medium,
  COUNT(*) AS sessions,
  SUM(has_landing) AS landing_sessions,
  SUM(has_crisis_page) AS crisis_sessions,
  SUM(has_call_click) AS call_click_sessions
FROM session_flags
GROUP BY source, medium
HAVING sessions >= 20
ORDER BY sessions DESC
"""

df = client.query(query).to_dataframe()
df["source_medium"] = df["source"] + " / " + df["medium"]
df.head()

In [ ]:
overall = pd.DataFrame(
    {
        "stage": ["Landing sessions", "Reached crisis page", "Clicked tel:131114"],
        "sessions": [
            int(df["landing_sessions"].sum()),
            int(df["crisis_sessions"].sum()),
            int(df["call_click_sessions"].sum()),
        ],
    }
)

fig = go.Figure(
    go.Funnel(
        y=overall["stage"],
        x=overall["sessions"],
        textinfo="value+percent initial",
    )
)
fig.update_layout(
    template="lifeline",
    title=f"Crisis Support Funnel (Last {DAYS_BACK} Days)",
)
lifeline_theme.add_lifeline_logo(fig)
fig.show()

In [ ]:
df["crisis_rate"] = df["crisis_sessions"] / df["landing_sessions"].replace(0, pd.NA)
df["call_rate_from_crisis"] = df["call_click_sessions"] / df["crisis_sessions"].replace(0, pd.NA)

channel_view = df[[
    "source_medium",
    "landing_sessions",
    "crisis_sessions",
    "call_click_sessions",
    "crisis_rate",
    "call_rate_from_crisis",
]].sort_values("landing_sessions", ascending=False)

channel_view.head(15)